## Week 2 - Pizza Runner

### <u>Introduction </u>

Did you know that over 115 million kilograms of pizza are devoured daily worldwide? This astounding statistic, coupled with Danny's serendipitous encounter with the phrase "80s Retro Styling and Pizza Is The Future!" on Instagram, sparked the inception of Pizza Runner. Recognizing that traditional pizza alone wouldn't suffice to propel his vision forward, Danny conceived the idea to "Uberize" pizza delivery—a concept that laid the foundation for Pizza Runner.

<div>
<img src="../sql-screenshots/pizza_runner_images/pizza-runner.png"/>
</div>


### <u>Entity-Relationship Diagram </u>
<div>
<img src="../sql-screenshots/pizza_runner_images/entity-diagram.png" width="500"/>
</div>


#### A. Pizza Metrics
##### 1. How many pizzas were ordered?
```sql
SELECT COUNT(pizza_id) AS pizza_count 
FROM pizza_runner.customer_orders
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s1.png" width="250"/>
</div>

##### 2. How many unique customer orders were made?
```sql 
SELECT COUNT(DISTINCT customer_id) AS unique_customer_orders 
FROM pizza_runner.customer_orders
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s2.png" width="250"/>
</div>

##### 3. How many successful orders were delivered by each runner?
```sql 
SELECT runner_id,
SUM(CASE 
  WHEN cancellation IS NULL OR 
       cancellation = 'null' OR 
       nullif(cancellation, '') IS NULL 
       THEN 1 ELSE 0 
  END) AS success_orders
FROM pizza_runner.runner_orders AS ro 
GROUP BY runner_id
ORDER BY runner_id 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s3.png" width="250"/>
</div>

##### 4. How many of each type of pizza was delivered?
```sql
SELECT pizza_name,
SUM(CASE 
  WHEN cancellation IS NULL OR 
       cancellation = 'null' OR 
       nullif(cancellation, '') IS NULL 
       THEN 1 ELSE 0 
  END) AS success_orders
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
LEFT JOIN pizza_runner.pizza_names AS pn 
ON co.pizza_id = pn.pizza_id 
GROUP BY pizza_name 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s4.png" width="250"/>
</div>

##### 5. How many Vegetarian and Meatlovers were ordered by each customer?


```sql
SELECT customer_id, pizza_name, COUNT(pizza_name) AS pizza_count 
FROM pizza_runner.customer_orders AS co 
LEFT JOIN pizza_runner.pizza_names AS pn 
ON co.pizza_id = pn.pizza_id
GROUP BY customer_id, pizza_name 
ORDER BY customer_id
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s5.png" width="250"/>
</div>

##### 6. What was the maximum number of pizzas delivered in a single order?


```sql 
WITH t1 AS (
SELECT ro.order_id, 
  COUNT(CASE WHEN cancellation IS NULL OR 
       cancellation = 'null' OR 
       nullif(cancellation, '') IS NULL 
       THEN 1 ELSE 0 
  END) AS pizza_success
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
GROUP BY ro.order_id 
)

SELECT order_id, MAX(pizza_success) AS max_pizza_order 
FROM t1 
GROUP BY order_id 
ORDER BY max_pizza_order DESC
LIMIT 1 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s6.png" width="250"/>
</div>

##### 7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?


```sql 
WITH t1 AS (
SELECT *, 
  CASE WHEN cancellation IS NULL 
  OR cancellation = 'null' 
  OR nullif(cancellation, '') IS NULL 
  THEN 1 ELSE 0 
  END AS cancel_status,
   CASE WHEN exclusions IS NULL 
            OR exclusions = 'null' 
            OR nullif(exclusions, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS exclusion_status, 
  CASE WHEN extras ='null' 
            OR nullif(extras, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS extras_status
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id
)

SELECT customer_id, SUM(change_count) AS total_change,SUM(no_change_count) AS total_no_change FROM 
(
  SELECT customer_id, 
      CASE WHEN exclusion_status = 1 AND extras_status = 1 
              OR exclusion_status = 1 AND extras_status = 0 
              OR exclusion_status = 0 AND extras_status = 1
              THEN 1 ELSE 0 END AS change_count,
      CASE WHEN exclusion_status = 0 AND extras_status = 0 
              THEN 1 ELSE 0 END as no_change_count
      FROM t1 
      WHERE cancel_status = 1 
)
GROUP BY customer_id 
ORDER BY customer_id 
```


<div>
<img src="../sql-screenshots/pizza_runner_images/s7.png" width="250"/>
</div>

##### 8. How many pizzas were delivered that had both exclusions and extras?


```sql 
WITH t1 AS (
SELECT *, 
  CASE WHEN cancellation IS NULL 
  OR cancellation = 'null' 
  OR nullif(cancellation, '') IS NULL 
  THEN 1 ELSE 0 
  END AS cancel_status,
   CASE WHEN exclusions IS NULL 
            OR exclusions = 'null' 
            OR nullif(exclusions, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS exclusion_status, 
  CASE WHEN extras ='null' 
            OR nullif(extras, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS extras_status
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id
)

SELECT pizza_id, 
      SUM(CASE WHEN exclusion_status = 1 AND extras_status = 1 
              THEN 1 ELSE 0 END) AS pizza_count
      FROM t1 
      WHERE cancel_status = 1
GROUP BY pizza_id 

```

<div>
<img src="../sql-screenshots/pizza_runner_images/s8.png" width="250"/>
</div>

##### 9. What was the total volume of pizzas ordered for each hour of the day?


```sql 
DROP TABLE IF EXISTS t1; 
CREATE TEMP TABLE t1 AS (
SELECT co.order_id, co.order_time, co.pizza_id, DATE_TRUNC('day', co.order_time) AS order_day, 
DATE_TRUNC('hour', co.order_time) AS order_hour
FROM pizza_runner.customer_orders AS co 
LEFT JOIN pizza_runner.runner_orders AS ro 
ON ro.order_id = co.order_id
);

SELECT DATE_PART('HOUR', order_hour) AS order_hour_part, COUNT(pizza_id) FROM t1 
GROUP BY order_hour_part
ORDER BY order_hour_part ASC
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s9.png" width="250"/>
</div>

##### 10. What was the volume of orders for each day of the week?


``` sql 
WITH t1 AS (
  SELECT
	TO_CHAR(order_time, 'Day') AS day,
	COUNT(order_id) AS order_count, 
	DATE_PART('dow', order_time) AS day_of_week 
FROM pizza_runner.customer_orders
GROUP BY order_time
ORDER BY order_time
)

SELECT day, SUM(order_count) AS total_orders
FROM t1 
GROUP BY day_of_week, day
ORDER BY day_of_week 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s10.png" width="250"/>
</div>

#### B. Runner And Customer Experience 
##### 1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)


```sql
SELECT DATE_TRUNC('week', "registration_date") :: DATE + 4 AS registration_week, 
COUNT(runner_id) AS runner_count
FROM pizza_runner.runners
GROUP BY registration_week
ORDER BY registration_week
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s11.png" width="250"/>
</div>

##### 2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

```sql 
WITH t1 AS (
SELECT co.order_id, ro.pickup_time, co.order_time,
   DATE_PART('minute', pickup_time::TIMESTAMP - order_time) AS time_taken
FROM pizza_runner.runner_orders AS ro 
INNER JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
WHERE pickup_time IS NOT NULL AND pickup_time != 'null' AND nullif(pickup_time, '') IS NOT NULL 
GROUP BY co.order_id,ro.pickup_time, co.order_time
)

SELECT FLOOR(AVG(time_taken)) AS average_time_taken FROM t1
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s12.png" width="250"/>
</div>

##### 3. Is there any relationship between the number of pizzas and how long the order takes to prepare?


```sql 
WITH t1 AS (
SELECT co.order_id, COUNT(co.order_id) AS pizza_count, 
DATE_PART('minute', ro.pickup_time::TIMESTAMP - co.order_time) AS prep_time
FROM pizza_runner.customer_orders AS co 
JOIN pizza_runner.runner_orders AS ro 
ON ro.order_id = co.order_id
WHERE pickup_time != 'null' AND pickup_time IS NOT NULL AND nullif(pickup_time, '') IS NOT NULL 
GROUP BY co.order_id, pickup_time, order_time 
)

SELECT pizza_count, AVG(prep_time) AS average_prep_time FROM t1 
GROUP BY pizza_count
ORDER BY pizza_count  
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s13.png" width="250"/>
</div>

##### 4. What was the average distance travelled for each customer?

```sql 
WITH cte_customer_order_distances AS (
SELECT 
  t1.customer_id,
  t1.order_id, t2.pickup_time, t2.cancellation, t2.duration, t1.order_time,
  REGEXP_REPLACE(t2.distance, '\s*km', '')::NUMERIC AS distance
FROM pizza_runner.customer_orders AS t1
INNER JOIN pizza_runner.runner_orders AS t2
  ON t1.order_id = t2.order_id
WHERE t2.pickup_time != 'null' AND t2.pickup_time IS NOT NULL 
)

SELECT customer_id, ROUND(AVG(distance),1) FROM cte_customer_order_distances
GROUP BY customer_id
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s14.png" width="250"/>
</div>

##### 5. What was the difference between the longest and shortest delivery times for all orders?


```sql 

WITH cte_customer_order_delivery AS (
SELECT 
  t1.order_id, t2.pickup_time, t2.cancellation, t1.order_time,
  REGEXP_REPLACE(duration, 'minutes|\s*mins|\sminute[s]?', '')::NUMERIC AS duration_min
FROM pizza_runner.customer_orders AS t1
INNER JOIN pizza_runner.runner_orders AS t2
  ON t1.order_id = t2.order_id
WHERE t2.pickup_time != 'null' AND t2.pickup_time IS NOT NULL 
)

SELECT MAX(duration_min) - MIN(duration_min) AS duration_diff
FROM cte_customer_order_delivery

```

<div>
<img src="../sql-screenshots/pizza_runner_images/s15.png" width="250"/>
</div>

##### 6. What was the average speed for each runner for each delivery and do you notice any trend for these values?

```sql 
DROP TABLE IF EXISTS ro_edited; 
CREATE TEMP TABLE ro_edited AS (
    SELECT customer_id, pizza_runner.customer_orders.order_id, 
      runner_id, 
      pickup_time, 
      REGEXP_REPLACE(distance, 'km.\s*km|km$', '')::NUMERIC AS distance_km, 
      REGEXP_REPLACE(duration, 'minutes|\s*mins|\sminute[s]?', '')::NUMERIC AS duration_min
FROM pizza_runner.runner_orders
LEFT JOIN pizza_runner.customer_orders
ON pizza_runner.runner_orders.order_id = pizza_runner.customer_orders.order_id
WHERE pickup_time != 'null'
);

WITH t2 AS (
SELECT customer_id, runner_id, DATE_PART('HOUR', pickup_time::TIMESTAMP) AS pickup_hour,
distance_km, duration_min, 
ROUND(distance_km / (duration_min / 60),2) AS speed_km_h 
FROM ro_edited
ORDER BY speed_km_h DESC, runner_id
)

SELECT  runner_id, customer_id, ROUND(AVG(speed_km_h),2) AS avg_speed FROM t2 
GROUP BY  runner_id, customer_id
ORDER BY runner_id, avg_speed DESC
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s16a.png" width="250"/>
</div>

##### 7. What is the successful delivery percentage for each runner?

```sql 
WITH cte AS 
(
	SELECT runner_id, order_id,
          CASE WHEN cancellation is NULL OR cancellation = 'null' OR nullif(cancellation, '') IS NULL 
          THEN 1
          ELSE 0 END AS sucess_delivery
    FROM pizza_runner.runner_orders
)
SELECT runner_id, ROUND( 100*SUM(sucess_delivery)/COUNT(order_id),2) AS success_rate
FROM cte
GROUP BY runner_id
ORDER BY runner_id
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s17.png" width="250"/>
</div>

#### C. Ingredient Optimisation 
##### 1. What are the standard ingredients for each pizza?

```sql 
WITH pizza_ingred AS (
SELECT pizza_id, 
      REGEXP_SPLIT_TO_TABLE(toppings, '[,\s]+')::INTEGER AS topping_id1
FROM pizza_runner.pizza_recipes AS pr 
)

SELECT
  pizza_id, 
  STRING_AGG(t2.topping_name::TEXT, ',') AS standard_ingredients
FROM pizza_ingred AS t1
INNER JOIN pizza_runner.pizza_toppings AS t2
  ON t1.topping_id1 = t2.topping_id
GROUP BY pizza_id
ORDER BY pizza_id;
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s18.png" width="600"/>
</div>

##### 2. What was the most commonly added extra?

```sql 
WITH extras_table AS (
SELECT pizza_id, REGEXP_SPLIT_TO_TABLE(extras, '\s*,\s*')::INTEGER AS extras_list 
FROM pizza_runner.customer_orders
WHERE extras IS NOT NULL AND extras != 'null' AND nullif(extras, '') IS NOT NULL 
)

SELECT topping_name, COUNT(*) AS topping_count 
FROM extras_table
INNER JOIN pizza_runner.pizza_toppings AS pt 
ON extras_table.extras_list = pt.topping_id
GROUP BY topping_name
ORDER BY topping_count DESC 

```

<div>
<img src="../sql-screenshots/pizza_runner_images/s19.png" width="250"/>
</div>

##### 3. What was the most common exclusion?


```sql 
WITH exclusion_table AS (
SELECT pizza_id, REGEXP_SPLIT_TO_TABLE(exclusions, '\s*,\s*')::INTEGER AS exclusions_list 
FROM pizza_runner.customer_orders
WHERE exclusions IS NOT NULL AND exclusions != 'null' AND nullif(exclusions, '') IS NOT NULL 
)

SELECT topping_name, COUNT(*) AS topping_count 
FROM exclusion_table
INNER JOIN pizza_runner.pizza_toppings AS pt 
ON exclusion_table.exclusions_list = pt.topping_id
GROUP BY topping_name
ORDER BY topping_count DESC 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s20.png" width="250"/>
</div>

##### 4. Generate an order item for each record in the customers_orders table in the format of one of the following:
      - Meat Lovers
      - Meat Lovers - Exclude Beef
      - Meat Lovers - Extra Bacon
      - Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers

```sql 
WITH cte_co_cleaned AS (
SELECT order_id, pizza_id, customer_id, order_time, 
CASE
  WHEN exclusions IN ('', 'null') THEN NULL
  ELSE exclusions
END AS exclusions,
CASE
  WHEN extras IN ('', 'null') THEN NULL
  ELSE extras
END AS extras,
ROW_NUMBER() OVER () AS original_row_number -- need this to maintain the order of rows. 
FROM pizza_runner.customer_orders
),
cte_extras_exclusions AS (
    SELECT
      order_id,
      customer_id,
      pizza_id,
      REGEXP_SPLIT_TO_TABLE(exclusions, '[,\s]+')::INTEGER AS exclusions_topping_id,
      REGEXP_SPLIT_TO_TABLE(extras, '[,\s]+')::INTEGER AS extras_topping_id,
      order_time,
      original_row_number
    FROM cte_co_cleaned
  UNION -- because you only want the distinct null rows that were removed by regexp_split_to_table
    SELECT
      order_id,
      customer_id,
      pizza_id,
      NULL AS exclusions_topping_id,
      NULL AS extras_topping_id,
      order_time,
      original_row_number
    FROM cte_co_cleaned
    WHERE exclusions IS NULL AND extras IS NULL  
  ), 
cte_complete_dataset AS ( -- combines the rows containing duplicate pizza_ids exclusion and extras into a single row 
  SELECT
    base.order_id,
    base.customer_id,
    base.pizza_id,
    names.pizza_name,
    base.order_time,
    base.original_row_number,
    STRING_AGG(exclusions.topping_name, ', ') AS exclusions,
    STRING_AGG(extras.topping_name, ', ') AS extras
  FROM cte_extras_exclusions AS base
  INNER JOIN pizza_runner.pizza_names AS names
    ON base.pizza_id = names.pizza_id
  LEFT JOIN pizza_runner.pizza_toppings AS exclusions
    ON base.exclusions_topping_id = exclusions.topping_id
  LEFT JOIN pizza_runner.pizza_toppings AS extras
    ON base.exclusions_topping_id = extras.topping_id
  GROUP BY
    base.order_id,
    base.customer_id,
    base.pizza_id,
    names.pizza_name,
    base.order_time,
    base.original_row_number
), 
cte_parsed_string_outputs AS (
SELECT
  order_id,
  customer_id,
  pizza_id,
  order_time,
  original_row_number,
  pizza_name,
  CASE WHEN exclusions IS NULL THEN '' ELSE ' - Exclude ' || exclusions END AS exclusions,
  CASE WHEN extras IS NULL THEN '' ELSE ' - Extra ' || exclusions END AS extras
FROM cte_complete_dataset
), 
final_output AS (
  SELECT
    order_id,
    customer_id,
    pizza_id,
    order_time,
    original_row_number,
    pizza_name || exclusions || extras AS order_item
  FROM cte_parsed_string_outputs
)
  
SELECT
  order_id,
  customer_id,
  pizza_id,
  order_time,
  order_item
FROM final_output
ORDER BY original_row_number;
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s21.png" width="900"/>
</div>

##### 5. Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients


```sql 
WITH cte_cleaned_table AS (
SELECT 
  order_id, 
  co.pizza_id, 
  customer_id, order_time,
  toppings as std_toppings, 
  CASE WHEN exclusions IN ('', 'null') THEN NULL ELSE exclusions 
  END AS exclusions, 
  CASE WHEN extras IN ('', 'null') THEN NULL ELSE extras
  END AS extras,
  ROW_NUMBER() OVER () AS original_row_number
FROM pizza_runner.customer_orders AS co 
LEFT JOIN pizza_runner.pizza_recipes AS pr 
ON co.pizza_id = pr.pizza_id
), 

-- create an exclusions table
cte_exclusions_table AS (
SELECT order_id, pizza_id, customer_id, order_time, original_row_number,  
REGEXP_SPLIT_TO_TABLE(exclusions, '[,\s]+')::INTEGER AS topping_id 
FROM cte_cleaned_table
), 

-- create an extras table
cte_extras_table AS (
SELECT order_id, pizza_id, customer_id, order_time, original_row_number, 
REGEXP_SPLIT_TO_TABLE(extras, '[,\s]+')::INTEGER AS topping_id 
FROM cte_cleaned_table
), 

-- create a standard base table
cte_base_table AS (
SELECT order_id, pizza_id, customer_id, order_time, original_row_number, 
REGEXP_SPLIT_TO_TABLE(std_toppings, '[,\s]+')::INTEGER AS topping_id 
FROM cte_cleaned_table
), 

-- create a base and exclusions and extras table 
cte_combined_orders AS (
  SELECT * FROM cte_base_table
  EXCEPT
  SELECT * FROM cte_exclusions_table
  UNION ALL
  SELECT * FROM cte_extras_table
), 

-- contains the combined orders with pizza name and topping names
cte_joined_table AS (
SELECT order_id, 
       cte_combined_orders.pizza_id,
       cte_combined_orders.customer_id, 
       order_time, 
      original_row_number, 
      pizza_name, 
      cte_combined_orders.topping_id, 
      topping_name 
      FROM cte_combined_orders
LEFT JOIN pizza_runner.pizza_names AS pn 
ON cte_combined_orders.pizza_id = pn.pizza_id
LEFT JOIN pizza_runner.pizza_toppings AS pt 
ON cte_combined_orders.topping_id = pt.topping_id
), 

-- contains a count of the toppings 
topping_pizza AS (
SELECT order_id, pizza_id, customer_id, order_time, original_row_number, pizza_name, topping_name, COUNT(topping_name) AS topping_count
FROM cte_joined_table
GROUP BY order_id, pizza_id, customer_id, order_time, original_row_number, pizza_name, topping_name
ORDER BY order_id, pizza_id, customer_id, pizza_name
)

-- final table
SELECT
  order_id,
  customer_id,
  pizza_id,
  order_time,
  original_row_number,  pizza_name || ': ' ||STRING_AGG(CASE WHEN topping_count > 1 THEN topping_count || 'x ' || topping_name ELSE topping_name END,',') AS ingredients_list
FROM topping_pizza
GROUP BY
  order_id,
  customer_id,
  pizza_id,
  order_time,
  original_row_number,
  pizza_name
ORDER BY   order_id,
  customer_id,
  original_row_number ASC
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s22.png" width="900"/>
</div>

##### 6. What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

#### 4. Part D. Pricing And Ratings 
##### 1. If a Meat Lovers pizza costs $12 and Vegetarian costs $10 and there were no charges for changes - how much money has Pizza Runner made so far if there are no delivery fees?

```sql 
WITH ro_edited AS (
SELECT 
 co.order_id, 
 runner_id, 
 pickup_time, 
 distance, 
 duration, pizza_name, 
  CASE WHEN 
  cancellation = 'null' OR cancellation IS NULL OR cancellation = '' THEN '0' ELSE '1'
  END as cancel_status
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
LEFT JOIN pizza_runner.pizza_names AS pn 
ON co.pizza_id = pn.pizza_id
)


SELECT 
SUM(CASE WHEN pizza_name = 'Meatlovers' THEN 12 ELSE 10 END) AS revenue
FROM ro_edited
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s23.png" width="150"/>
</div>

##### 2. What if there was an additional $1 charge for any pizza extras? + Add cheese is $1 extra

```sql 
WITH cte_cleaned_customer_orders AS (
  SELECT
    order_id,
    customer_id,
    pizza_id,
    CASE
      WHEN exclusions IN ('', 'null') THEN NULL
      ELSE exclusions
    END AS exclusions,
    CASE
      WHEN extras IN ('', 'null') THEN NULL
      ELSE extras
    END AS extras,
    order_time,
    ROW_NUMBER() OVER () AS original_row_number
  FROM pizza_runner.customer_orders
  WHERE EXISTS (
    SELECT 1 FROM pizza_runner.runner_orders
    WHERE runner_orders.order_id = customer_orders.order_id
      AND runner_orders.pickup_time != 'null'
  )
), cost_table AS (
SELECT pizza_id, CASE WHEN pizza_id = 1 THEN 12 ELSE 10 END AS pizza_cost, 
COALESCE(
      CARDINALITY(REGEXP_SPLIT_TO_ARRAY(extras, '[,\s]+')),
      0
    ) AS extras_cost
FROM cte_cleaned_customer_orders
)

SELECT SUM(COALESCE(pizza_cost,0) + COALESCE(extras_cost,0)) AS revenue 
FROM cost_table; 


```

<div>
<img src="../sql-screenshots/pizza_runner_images/s24.png" width="150"/>
</div>

##### 3. The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.

```sql 
SELECT SETSEED(1);

DROP TABLE IF EXISTS pizza_runner.ratings;
CREATE TABLE pizza_runner.ratings (
  "order_id" INTEGER,
  "rating" INTEGER
);
-- only non-cancelled orders are given a rating
INSERT INTO pizza_runner.ratings
SELECT
  order_id,
  FLOOR(1 + 5 * RANDOM()) AS rating
FROM pizza_runner.runner_orders
WHERE pickup_time IS NOT NULL;
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s25.png" width="150"/>
</div>

##### 4. Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries? + customer_id + order_id + runner_id + rating + order_time + pickup_time + Time between order and pickup + Delivery duration + Average speed + Total number of pizzas

```sql 
SELECT SETSEED(1);

DROP TABLE IF EXISTS pizza_runner.ratings;
CREATE TABLE pizza_runner.ratings (
  "order_id" INTEGER,
  "rating" INTEGER
);

INSERT INTO pizza_runner.ratings
SELECT
  order_id,
  FLOOR(1 + 5 * RANDOM()) AS rating
FROM pizza_runner.runner_orders
WHERE pickup_time IS NOT NULL;

WITH t1 AS (
SELECT co.order_id, runner_id, rating, order_time, pickup_time, 
  REGEXP_REPLACE(distance, 'km.\s*km|km$', '')::NUMERIC AS distance, 
  REGEXP_REPLACE(duration, 'minutes|\s*mins|\sminute[s]?', ''):: NUMERIC AS duration, 
DATE_PART('min', pickup_time::TIMESTAMP - order_time) AS pickup_minutes
FROM pizza_runner.ratings AS r 
INNER JOIN pizza_runner.runner_orders AS ro 
ON r.order_id = ro.order_id
INNER JOIN pizza_runner.customer_orders AS co 
ON co.order_id = r.order_id
WHERE pickup_time != 'null' OR cancellation NOT IN ('Restaurant Cancellation', 'Customer Cancellation')
), pizza_count AS (
SELECT order_id, COUNT(order_id) AS pizza_count FROM t1
GROUP BY order_id
ORDER BY order_id
), runner_speed AS (
SELECT order_id,runner_id, ROUND(AVG(distance / (duration / 60)),1) AS avg_speed FROM t1
GROUP BY order_id, runner_id
)

SELECT 
 DISTINCT(t1.order_id), 
 t1.runner_id, 
 rating, 
 order_time, 
 pickup_time, 
 pickup_minutes, 
 avg_speed, pizza_count
FROM t1
INNER JOIN runner_speed AS rs
ON t1.order_id = rs.order_id AND t1.runner_id = rs.runner_id 
INNER JOIN pizza_count AS pc 
ON t1.order_id = pc.order_id 
ORDER BY order_id
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s26a.png" width="750"/>
</div>

##### 5. If a Meat Lovers pizza was $12 and Vegetarian $10 fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled - how much money does Pizza Runner have left over after these deliveries?

```sql 
WITH ro_edited AS (
SELECT 
 co.order_id, 
 runner_id, 
 pickup_time, 
 distance, 
 duration, pizza_name, 
  CASE WHEN 
  cancellation = 'null' OR cancellation IS NULL OR cancellation = '' THEN '0' ELSE '1'
  END as cancel_status
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
LEFT JOIN pizza_runner.pizza_names AS pn 
ON co.pizza_id = pn.pizza_id
), t1 AS (
SELECT 
 order_id, 
 runner_id, 
 pickup_time, 
 REGEXP_REPLACE(distance, 'km.\s*km|km$', '')::NUMERIC * 0.3 AS distance_cost, 
 CASE WHEN pizza_name = 'Meatlovers' THEN 12 ELSE 10 END AS revenue
FROM ro_edited
WHERE pickup_time != 'null' AND cancel_status != '1'
)

SELECT SUM(revenue) - SUM(distance_cost) AS leftover_revenue FROM t1; 
```


<div>
<img src="../sql-screenshots/pizza_runner_images/s27.png" width="250"/>
</div>